<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#9.1-复杂数据的局部性建模" data-toc-modified-id="9.1-复杂数据的局部性建模-1">9.1 复杂数据的局部性建模</a></span></li><li><span><a href="#9.2-连续型和离散型特征的树的模型的构建" data-toc-modified-id="9.2-连续型和离散型特征的树的模型的构建-2">9.2 连续型和离散型特征的树的模型的构建</a></span></li><li><span><a href="#9.3-将CART算法用于回归" data-toc-modified-id="9.3-将CART算法用于回归-3">9.3 将CART算法用于回归</a></span></li><li><span><a href="#9.3.1-构建树" data-toc-modified-id="9.3.1-构建树-4">9.3.1 构建树</a></span><ul class="toc-item"><li><span><a href="#9.3.2-运行代码" data-toc-modified-id="9.3.2-运行代码-4.1">9.3.2 运行代码</a></span></li></ul></li><li><span><a href="#9.4-树剪枝" data-toc-modified-id="9.4-树剪枝-5">9.4 树剪枝</a></span><ul class="toc-item"><li><span><a href="#9.4.1-预剪枝" data-toc-modified-id="9.4.1-预剪枝-5.1">9.4.1 预剪枝</a></span></li><li><span><a href="#New-heading" data-toc-modified-id="New-heading-5.2">New heading</a></span></li></ul></li><li><span><a href="#9.6-示例：树回归与标准回归的比较" data-toc-modified-id="9.6-示例：树回归与标准回归的比较-6">9.6 示例：树回归与标准回归的比较</a></span></li></ul></div>

# 树回归
现实生活中，很多问题都是非线性的，不可能使用全局现行模型来拟合任何数据，一种可行的方法是将数据集切分成很多块易建模的数据，然后利用前一章的线性回归技术来建模。

## 9.1 复杂数据的局部性建模
决策树不断地将数据切分成小的数据集，直到所有目标变量完全相同，或者数据数据不能再切分为止，决策树是一种贪心算法，它要在给定时间内作出最佳选择，但并不关心能否达到全局最优。

- 优点：可以对复杂和非线性的数据建模
- 缺点：结果不易理解
- 适用数据类型：数值型和标称型数据

二元切分法：每次把数据集切分成两份，如果数据的某特征值等于切分所要求的值，那么这些数据就进入树的左子树，反之则进入树的右子树。

二元切分法易于对树构建过程进行调整以处理连续型特征，具体处理方法是：如果特征值大于给定值就走左子树，否则就走右子树。CART稍作修改能够处理回归问题。

回归树和分类树的思路类似，但叶节点的数据类型不是离散型，而是连续型。

1. 收集数据：采用任意方法收集数据
2. 准备数据：需要数值型的数据，标称型的数据应该映射成二值型数据
3. 分析数据：绘出数据的二维可视化显示结果，以字典方式生成树
4. 训练算法：大部分时间都花费在叶节点树模型构建上
5. 测试算法：使用测试数据集上的$R^2$值来分析模型效果
6. 使用模型：使用训练出的树做预测

## 9.2 连续型和离散型特征的树的模型的构建

1. 使用字典存储树的结构
2. 面向对象构造树节点
```python
class treeNode():
    def __init__(self, feat, val, right, left):
        featureToSplitOn = feat
        valueOfSplit = val
        rightBranch = right
        leftBranch = left
```
createTree()伪代码：

    找到最佳待切分特征：
        如果该节点不能再分，将该节点存为叶节点
        执行二元切分
        在右子树调用createTree()方法
        在左子树调用createTree()方法

In [1]:
# CART算法
from numpy import *
def loadDataSet(fileName):
    '''读取数据
    '''
    dataMat = []
    fr = open(fileName)
    for line in fr.readlines():
        curLine = line.strip().split('\t')
        fltLine = map(float, curLine)    # 将每行映射成浮点数
        dataMat.append(fltLine)
    return dataMat

def binSplitDataSet(dataSet, feature, value):
    '''二元切分
    传入：
    dataSet：数据矩阵
    feature：指定列
    value：指定特征值
    返回：
    切分后的子矩阵
    '''
    mat0 = dataSet[nonzero(dataSet[:,feature] > value)[0], :]
    mat1 = dataSet[nonzero(dataSet[:,feature] <= value)[0], :]
    return mat0, mat1

# 工具
def regLeaf(dataSet):
    '''获取叶节点的模型，在回归树中，数据集的均值
    '''
    return mean(dataSet[:, -1])

def regErr(dataSet):
    '''计算数据集的总方差
    返回：
    方差×样本数
    '''
    return var(dataSet[:, -1]) * shape(dataSet)[0]

def createTree(dataSet, leafType=regLeaf, errType=regErr, ops=(1, 4)):
    '''构建树
    传入：
    leafType：计算叶节点方法
    errType：总方差误差方法
    ops：指定操作，1和4两种
    '''
    feat, val = chooseBestSplit(dataSet, leafType, errType, ops)   # 回归树切分函数， 返回特征和特征值
    if feat == None:    # 可切分的特征为空，返回叶节点值
        return val
    retTree = {}    # 回归树字典
    retTree['spInd'] = feat    # 切分的特征
    retTree['spVal'] = val    # 切分的值
    lSet, rSet = binSplitDataSet(dataSet, feat, val)    # 二元切分， 返回俩子数据矩阵
    retTree['left'] = createTree(lSet, leafType, errType, ops)
    retTree['right'] = createTree(rSet, leafType, errType, ops)
    return retTree
        

In [2]:
testMat = mat((eye(4)))
mat0, mat1 = binSplitDataSet(testMat, 1, 0.5)    # 按第1+1 列划分数据集
print mat0, mat1

[[ 0.  1.  0.  0.]] [[ 1.  0.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]


## 9.3 将CART算法用于回归
为了成功构建以分段常数为叶节点的树，需要度量出数据的一致性。使用每条数据对于全体数据的均值的标准差或者方差。
## 9.3.1 构建树
chooseBestSplit()——给定某个误差计算方法，该函数会找到数据集上最佳二元切分方式。当树停止划分的时候就会返回一个叶子节点。
该函数的为代码大致如下：

    对每个特征：
        对每个特征：
            将数据集分为两个部分
            计算切分误差
            如果当前误差小于最小误差，那么将当前且分设置为最佳切分，并更新最小误差
    返回最佳切分特征和阈值

In [3]:
# 回归树切分函数
def chooseBestSplit(dataSet, leafType=regLeaf, errType=regErr, ops=(1, 4)):
    '''创建切分点，有三种情况不切法数据集
    传入：
    dataSet：数据矩阵
    leafType：创建叶节点方法
    errType：计算误差方式
    '''
    tolS = ops[0]    # 误差下降容忍度
    tolN = ops[1]    # 切分最少样本数量
    # 如果所有值相等，则退出
    if len(set(dataSet[:,-1].T.tolist()[0])) == 1:    # 第一个不切分情况：所有类标一致
        return None, leafType(dataSet)
    m, n = shape(dataSet)
    S = errType(dataSet)    # 计算数据集样本总方差
    bestS, bestIndex, bestValue = inf, 0, 0    # 初始化
    for featIndex in range(n-1):    # 遍历所有属性
        for splitVal in set(dataSet[:, featIndex].flat):    # 修复matix不能使用set()的bug
            mat0, mat1 = binSplitDataSet(dataSet, featIndex, splitVal)
            if (shape(mat0)[0] < tolN) or (shape(mat1)[0] < tolN):    # 子矩阵小于切分最少样本，跳出for
                continue
            newS = errType(mat0) + errType(mat1)
            if newS < bestS:    # 更新最小的误差
                bestIndex = featIndex
                bestValue = splitVal
                bestS = newS
    # 第二个不切分情况：如果误差减少不大，则退出
    if abs(S - bestS) < tolS:
        return None, leafType(dataSet)
    mat0, mat1 = binSplitDataSet(dataSet, bestIndex, bestValue)
    # 第三个不切分情况：如果切分出来的数据很小，则退出
    if (shape(mat0)[0] < tolN) or (shape(mat1)[0] < tolN):
        return None, leafType(dataSet)
#     print bestIndex, bestValue
    return bestIndex, bestValue

### 9.3.2 运行代码


In [4]:
# 测试数据一
myDat = loadDataSet('ex00.txt')
myMat = mat(myDat)
printInf = createTree(myMat)
print printInf

{'spInd': 0, 'spVal': 0.48813000000000001, 'right': -0.044650285714285719, 'left': 1.0180967672413792}


In [5]:
# 测试数据二
myDat1 = loadDataSet('ex0.txt')
myMat1 = mat(myDat1)
printInf = createTree(myMat1)
printInf

{'left': {'left': {'left': 3.9871631999999999,
   'right': 2.9836209534883724,
   'spInd': 1,
   'spVal': 0.79758300000000004},
  'right': 1.980035071428571,
  'spInd': 1,
  'spVal': 0.58200200000000002},
 'right': {'left': 1.0289583666666666,
  'right': -0.023838155555555553,
  'spInd': 1,
  'spVal': 0.19783400000000001},
 'spInd': 1,
 'spVal': 0.39434999999999998}

## 9.4 树剪枝
在chooseBestSplit()提前中止，通过修改ops参数达到预剪枝效果。

通过使用测试集和训练集，称为后剪枝

### 9.4.1 预剪枝
修改ops里的误差容忍度和最大叶节点样本数目，能够控制叶节点数目。如下，当误差容忍度提高，叶节点的数目变少。

In [6]:
myMat2 = mat(loadDataSet('ex2.txt'))

createTree(myMat2, ops=(10000, 4))

{'left': 101.35815937735848,
 'right': -2.6377193297872341,
 'spInd': 0,
 'spVal': 0.49917099999999998}

### New heading

In [7]:
def isTree(obj):
    '''判断是否是一颗树
    '''
    return (type(obj).__name__=='dict')

def getMean(tree):
    '''获取整棵树的均值
    '''
    if isTree(tree['right']): 
        tree['right'] = getMean(tree['right'])
    if isTree(tree['left']): 
        tree['left'] = getMean(tree['left'])
    return (tree['left']+tree['right'])/2.0
    
def prune(tree, testData):
    '''树剪枝方法
    传入：
    tree：一棵树
    testData：剪枝所需的测试数据
    返回：剪枝过后的树
    '''
    if shape(testData)[0] == 0:     # 没有测试数据，对树进行塌陷处理
        return getMean(tree)
    if (isTree(tree['right']) or isTree(tree['left'])):    # 左右分支不是树，则进行修建
        lSet, rSet = binSplitDataSet(testData, tree['spInd'], tree['spVal'])
    if isTree(tree['left']):    # 
        tree['left'] = prune(tree['left'], lSet)
    if isTree(tree['right']): 
        tree['right'] =  prune(tree['right'], rSet)
    # 都是叶节点的情况，查看是否需要合并
    if not isTree(tree['left']) and not isTree(tree['right']):
        lSet, rSet = binSplitDataSet(testData, tree['spInd'], tree['spVal'])
        errorNoMerge = sum(power(lSet[:,-1] - tree['left'],2)) +\
            sum(power(rSet[:,-1] - tree['right'],2))
        treeMean = (tree['left']+tree['right'])/2.0
        errorMerge = sum(power(testData[:,-1] - treeMean,2))
        if errorMerge < errorNoMerge: 
            print "merging"
            return treeMean
        else: return tree
    else: return tree

In [8]:
# 创建最大的树
myTree = createTree(myMat2, ops=(0,1))
# 导入数据
myMat2Test = mat(loadDataSet('ex2test.txt'))
prune(myTree, myMat2Test)

merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging


{'left': {'left': {'left': {'left': 92.523991499999994,
    'right': {'left': {'left': {'left': 112.386764,
       'right': 123.559747,
       'spInd': 0,
       'spVal': 0.96039799999999997},
      'right': 135.83701300000001,
      'spInd': 0,
      'spVal': 0.95851200000000003},
     'right': 111.2013225,
     'spInd': 0,
     'spVal': 0.956951},
    'spInd': 0,
    'spVal': 0.96596899999999997},
   'right': {'left': {'left': {'left': {'left': {'left': {'left': {'left': {'left': {'left': {'left': {'left': 96.41885225,
              'right': 69.318648999999994,
              'spInd': 0,
              'spVal': 0.94882200000000005},
             'right': {'left': {'left': 110.03503850000001,
               'right': {'left': 65.548417999999998,
                'right': {'left': 115.75399400000001,
                 'right': {'left': {'left': 94.396114499999996,
                   'right': 85.005351000000005,
                   'spInd': 0,
                   'spVal': 0.912161},
          

## 9.6 示例：树回归与标准回归的比较

In [9]:
def regTreeEval(model, inDat):
    '''返回一个浮点数据集'''
    return float(model)

def modelTreeEval(model, inDat):
    n = shape(inDat)[1]
    X = mat(ones((1,n+1)))
    X[:,1:n+1]=inDat
    return float(X*model)

def treeForeCast(tree, inData, modelEval=regTreeEval):
    '''给出一个回归的预测值'''
    if not isTree(tree): 
        return modelEval(tree, inData)
    if inData[tree['spInd']] > tree['spVal']:
        if isTree(tree['left']): 
            return treeForeCast(tree['left'], inData, modelEval)
        else: 
            return modelEval(tree['left'], inData)
    else:
        if isTree(tree['right']): 
            return treeForeCast(tree['right'], inData, modelEval)
        else: 
            return modelEval(tree['right'], inData)
        
def createForeCast(tree, testData, modelEval=regTreeEval):
    '''自顶向下遍历树'''
    m=len(testData)
    yHat = mat(zeros((m,1)))
    for i in range(m):
        yHat[i,0] = treeForeCast(tree, mat(testData[i]), modelEval)
    return yHat

In [10]:
# from numpy import *

# from Tkinter import *
# import regTrees

# import matplotlib
# matplotlib.use('TkAgg')
# from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
# from matplotlib.figure import Figure

# def reDraw(tolS,tolN):
#     reDraw.f.clf()        # clear the figure
#     reDraw.a = reDraw.f.add_subplot(111)
#     if chkBtnVar.get():
#         if tolN < 2: tolN = 2
#         myTree=regTrees.createTree(reDraw.rawDat, regTrees.modelLeaf,\
#                                    regTrees.modelErr, (tolS,tolN))
#         yHat = regTrees.createForeCast(myTree, reDraw.testDat, \
#                                        regTrees.modelTreeEval)
#     else:
#         myTree=regTrees.createTree(reDraw.rawDat, ops=(tolS,tolN))
#         yHat = regTrees.createForeCast(myTree, reDraw.testDat)
#     reDraw.a.scatter(reDraw.rawDat[:,0], reDraw.rawDat[:,1], s=5) #use scatter for data set
#     reDraw.a.plot(reDraw.testDat, yHat, linewidth=2.0) #use plot for yHat
#     reDraw.canvas.show()
    
# def getInputs():
#     try: tolN = int(tolNentry.get())
#     except: 
#         tolN = 10 
#         print "enter Integer for tolN"
#         tolNentry.delete(0, END)
#         tolNentry.insert(0,'10')
#     try: tolS = float(tolSentry.get())
#     except: 
#         tolS = 1.0 
#         print "enter Float for tolS"
#         tolSentry.delete(0, END)
#         tolSentry.insert(0,'1.0')
#     return tolN,tolS

# def drawNewTree():
#     tolN,tolS = getInputs()#get values from Entry boxes
#     reDraw(tolS,tolN)
    
# root=Tk()

# reDraw.f = Figure(figsize=(5,4), dpi=100) #create canvas
# reDraw.canvas = FigureCanvasTkAgg(reDraw.f, master=root)
# reDraw.canvas.show()
# reDraw.canvas.get_tk_widget().grid(row=0, columnspan=3)

# Label(root, text="tolN").grid(row=1, column=0)
# tolNentry = Entry(root)
# tolNentry.grid(row=1, column=1)
# tolNentry.insert(0,'10')
# Label(root, text="tolS").grid(row=2, column=0)
# tolSentry = Entry(root)
# tolSentry.grid(row=2, column=1)
# tolSentry.insert(0,'1.0')
# Button(root, text="ReDraw", command=drawNewTree).grid(row=1, column=2, rowspan=3)
# chkBtnVar = IntVar()
# chkBtn = Checkbutton(root, text="Model Tree", variable = chkBtnVar)
# chkBtn.grid(row=3, column=0, columnspan=2)

# reDraw.rawDat = mat(regTrees.loadDataSet('sine.txt'))
# reDraw.testDat = arange(min(reDraw.rawDat[:,0]),max(reDraw.rawDat[:,0]),0.01)
# reDraw(1.0, 10)
               
# root.mainloop()